# The Battle of Neighbourhood in Ghana

## The Objective is to analyse the various venues around an area in order to make a better choice when one is relocating or exploring a new city in Accra

# Problem Statement

## One of the primary difficulties you may face while relocating to a completely new place in Accra is finding a suitable place with shops near you within 1 to 5 KM. 


## Finding a suitable place that is near bus-stop, boutiques, bars, and markets is very challenging.

Let's Get our hands dirty

# 1-Import and Install all the required library

In [3]:
#Install libraries
!pip install folium==0.5.0  # uncomment this line if you haven't completed the Foursquare API lab
print('Folium installed')

!pip install  geopy 
print('geopy installed')

#import libraries
import requests # library to handle requests
import pandas as pd
import numpy as np
import folium # map rendering library
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim


#set dtaframe properties
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


print('Libraries imported.')

     |████████████████████████████████| 81kB 7.2MB/s eta 0:00:011
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
Folium installed
geopy installed
Libraries imported.


# 2-Add Foursquare API  Details(Not to be used in Production)

In [108]:
CLIENT_ID = '***********************' # your Foursquare ID
CLIENT_SECRET = '***********************' # your Foursquare Secret
VERSION = '***********************'
LIMIT = 30
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

# 3-Exploring Accra

#### We import the data from csv  (it is difficult to get the city coordinates online for Ghana. Therefore I tried  get the coordinates online  and use dropbox to make the url of the csv public. )

In [11]:
#Get table data from link
df=pd.read_csv('https://uc4136f6b6bf4b385bbba776527b.dl.dropboxusercontent.com/cd/0/get/A9_UBd4D_5BzT2OeS6-bTghI_taC6nn_uEWxxxdHtQPRoyktE5gWVqOJ18vFsy5O-5X00GyFlBJkNrtHk-8g1E5cq17VzdNZJYS5bWWKXKMnFXdpQwlC_5JKNvyULKXk-kU/file?dl=1#')
df
#

,City,Latitude,Longitude
0,Cantonments,5.578107,-0.181827
1,Nungua,5.606920,-0.087268
2,Tesano,5.605129,-0.241019
3,East Legon,5.636392,-0.162693
4,Lapaz,5.611948,-0.255141
5,Dansoman,5.548134,-0.260028
6,Mataheko,5.566418,-0.255980
7,Berekuso,5.756544,-0.231170
8,Amasaman,5.706725,-0.305364
9,Ridge,5.563587,-0.195976


In [109]:
#Build a function to get Nearby Places withing  Radius from each city.

def getNearbyPlaces(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius,100)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name,lat, lng, v['venue']['name'], v['venue']['location']['lat'],  v['venue']['location']['lng'], v['venue']['categories'][0]['name'] ,v['venue']['location']['distance'],v['venue']['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Places', 'Places Latitude', 'Places Longitude',  'Found Places', 'Latitudes', 'Longitudes', 'Places Category','Distance','id']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

We now compute all places within 5 km Radius 

In [126]:
getNear = getNearbyPlaces(names=df['City'],latitudes=df['Latitude'], longitudes=df['Longitude'],radius=5000 )
getNear.head()

#you can use getNearbyPlaces to get full list of various Places

Found 743 venues in 14 neighborhoods.


,Places,Places Latitude,Places Longitude,Found Places,Latitudes,Longitudes,Places Category,Distance,id
0,Cantonments,5.578107,-0.181827,La Villa Boutique Hotel,5.567260,-0.183356,Hotel,1219,4ea28fde0aaf711180a74c1f
1,Cantonments,5.578107,-0.181827,Fiesta Residence,5.575963,-0.185265,Modern European Restaurant,449,55ddb5a2498e3bb48ca6b2e2
2,Cantonments,5.578107,-0.181827,Bistro 22,5.571572,-0.169086,Bistro,1587,539477fc498edbdef86d43e5
3,Cantonments,5.578107,-0.181827,Restaurant Chez Clarisse Mama Africa,5.564269,-0.183821,African Restaurant,1556,4fd90666e4b071d9bd1f7e6f
4,Cantonments,5.578107,-0.181827,The Honeysuckle,5.568717,-0.184842,Pub,1097,4be381c32fc7d13a2e8f083a


We check the count of places nearby each city 

In [14]:
newdata= getNear.groupby('Places').count()
newdata=newdata.sort_values(by='Found Places')
newdata

,Places Latitude,Places Longitude,Found Places,Places Latitude,Places Longitude,Places Category,Distance,id
Places,,,,,,,,
Berekuso,2,2,2,2,2,2,2,2
Amasaman,4,4,4,4,4,4,4,4
Oyibi,5,5,5,5,5,5,5,5
Dansoman,28,28,28,28,28,28,28,28
Nungua,30,30,30,30,30,30,30,30
Lapaz,37,37,37,37,37,37,37,37
Mataheko,49,49,49,49,49,49,49,49
Madina,58,58,58,58,58,58,58,58
Tesano,59,59,59,59,59,59,59,59


We now Explore those places based on their category

In [125]:
newdata2= getNear.groupby('Places Category').count()
newdata2=newdata2.sort_values(by='Found Places', ascending=False)
newdata2.head()

,Places,Places Latitude,Places Longitude,Found Places,Latitudes,Longitudes,Distance,id
Places Category,,,,,,,,
Hotel,60,60,60,60,60,60,60,60
African Restaurant,50,50,50,50,50,50,50,50
Restaurant,44,44,44,44,44,44,44,44
Fast Food Restaurant,42,42,42,42,42,42,42,42
Shopping Mall,27,27,27,27,27,27,27,27


# Now,Lets compare Dansoman and Cantonment  based on  nearby venues ratings



We compute the ID for each places because the Foursquare place uses this input to get the Rating 

In [127]:
new_dataset_se = getNear[['Places','id','Distance','Found Places','Places Category']].copy()
new_dataset_se.head()

,Places,id,Distance,Found Places,Places Category
0,Cantonments,4ea28fde0aaf711180a74c1f,1219,La Villa Boutique Hotel,Hotel
1,Cantonments,55ddb5a2498e3bb48ca6b2e2,449,Fiesta Residence,Modern European Restaurant
2,Cantonments,539477fc498edbdef86d43e5,1587,Bistro 22,Bistro
3,Cantonments,4fd90666e4b071d9bd1f7e6f,1556,Restaurant Chez Clarisse Mama Africa,African Restaurant
4,Cantonments,4be381c32fc7d13a2e8f083a,1097,The Honeysuckle,Pub


In [122]:
new_dataset_se=new_dataset_se[ new_dataset_se['Distance']< 1000 ]
new_dataset_se.head()

,Places,id,Distance,Found Places,Places Category
1,Cantonments,55ddb5a2498e3bb48ca6b2e2,449,Fiesta Residence,Modern European Restaurant
6,Cantonments,4cd4b6947da9a35dc27be5b9,649,Rockstone's Office,Nightclub
32,Cantonments,4f46b964e4b0e7e90cab77af,882,Toro Tapas Bar,Spanish Restaurant
81,Cantonments,4dc6ea8be4cd169dc6717888,898,Thai Island,Thai Restaurant
149,Tesano,4e465971cc3f7ae3e1ac52a0,715,Las palmas,African Restaurant


We want to analyse only 2 places because the Quota for Foursquare API is limited for the endpoint responsible for the rating (It is forPremium Calls)

In [60]:
average_dansoman=new_dataset_se[ new_dataset_se['Places'] =="Dansoman" ]
average_dansoman

,Places,id,Distance,Found Places,Places Category
326,Dansoman,4f91c0e4e4b04fef00e2f4fb,471,JD PIzza Bazaar,Pub
333,Dansoman,4e8b9a3129c22b68aaef0c38,586,Tipsy Pub,Bar
340,Dansoman,4bf7791f508c0f471dad3d31,466,Maximum Gym,Gym / Fitness Center
342,Dansoman,4de6d84dd164402a0f1b4c6f,975,King Solomon,Bar


In [77]:
average_cantonment=new_dataset_se[ new_dataset_se['Places'] =="Cantonments" ]
average_cantonment

,Places,id,Distance,Found Places,Places Category
1,Cantonments,55ddb5a2498e3bb48ca6b2e2,449,Fiesta Residence,Modern European Restaurant
6,Cantonments,4cd4b6947da9a35dc27be5b9,649,Rockstone's Office,Nightclub
32,Cantonments,4f46b964e4b0e7e90cab77af,882,Toro Tapas Bar,Spanish Restaurant
81,Cantonments,4dc6ea8be4cd169dc6717888,898,Thai Island,Thai Restaurant


# You may want to find out places around those places for your daily needs

Lets Compare Dansoman and Cantonment since both have almost similar count of places within 1km Radius

In [74]:
#Build a function to get Nearby Places withing 500 Rdius from each city.

def getNearbyPlacesRate(names):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id='***********************'&client_secret='***********************'&v='***********************''.format(names)
        # make the GET request
    print(names)
    results = requests.get(url).json()["response"]['venue']['rating']
        
    return(results)

In [75]:
#Awoshie Abrantie Coordinates latitudes=5.5973674 longitudes=-0.2558485
#this is a unit test just to test the function
getNearbyPlacesRate( "4ea28fde0aaf711180a74c1f")

4ea28fde0aaf711180a74c1f


8.7

In [76]:
average_dansoman['rating'] = average_dansoman['id'].apply(getNearbyPlacesRate)
average_dansoman

4f91c0e4e4b04fef00e2f4fb
4e8b9a3129c22b68aaef0c38
4bf7791f508c0f471dad3d31
4de6d84dd164402a0f1b4c6f


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Places,id,Distance,Found Places,Places Category,rating
326,Dansoman,4f91c0e4e4b04fef00e2f4fb,471,JD PIzza Bazaar,Pub,6.7
333,Dansoman,4e8b9a3129c22b68aaef0c38,586,Tipsy Pub,Bar,6.0
340,Dansoman,4bf7791f508c0f471dad3d31,466,Maximum Gym,Gym / Fitness Center,5.7
342,Dansoman,4de6d84dd164402a0f1b4c6f,975,King Solomon,Bar,5.6


In [78]:
average_cantonment['rating'] = average_cantonment['id'].apply(getNearbyPlacesRate)
average_cantonment

55ddb5a2498e3bb48ca6b2e2
4cd4b6947da9a35dc27be5b9
4f46b964e4b0e7e90cab77af
4dc6ea8be4cd169dc6717888


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Places,id,Distance,Found Places,Places Category,rating
1,Cantonments,55ddb5a2498e3bb48ca6b2e2,449,Fiesta Residence,Modern European Restaurant,8.0
6,Cantonments,4cd4b6947da9a35dc27be5b9,649,Rockstone's Office,Nightclub,7.6
32,Cantonments,4f46b964e4b0e7e90cab77af,882,Toro Tapas Bar,Spanish Restaurant,7.2
81,Cantonments,4dc6ea8be4cd169dc6717888,898,Thai Island,Thai Restaurant,6.7


We use describe to analyse the rating such the average,min and the max of Cantonments nearby place

In [81]:
average_cantonment_rate=average_cantonment['rating']
average_cantonment_rate.describe()

count    4.000000
mean     7.375000
std      0.556028
min      6.700000
25%      7.075000
50%      7.400000
75%      7.700000
max      8.000000
Name: rating, dtype: float64

We use describe to analyse the rating such the average,min and the max of Dansoman nearby place

In [86]:
average_dansoman_rate=average_dansoman['rating']
average_dansoman_rate.describe()

count    4.000000
mean     6.000000
std      0.496655
min      5.600000
25%      5.675000
50%      5.850000
75%      6.175000
max      6.700000
Name: rating, dtype: float64

# Now lets Visualize the places around each city within 5km Radius

In [87]:
#import folium library
import folium # map rendering library


address = 'Dansoman'

#set Map boundaries
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 5.5468112, -0.2656595.


In [123]:
getNear.head()

,Places,Places Latitude,Places Longitude,Found Places,Latitudes,Longitudes,Places Category,Distance,id
0,Cantonments,5.578107,-0.181827,La Villa Boutique Hotel,5.567260,-0.183356,Hotel,1219,4ea28fde0aaf711180a74c1f
1,Cantonments,5.578107,-0.181827,Fiesta Residence,5.575963,-0.185265,Modern European Restaurant,449,55ddb5a2498e3bb48ca6b2e2
2,Cantonments,5.578107,-0.181827,Bistro 22,5.571572,-0.169086,Bistro,1587,539477fc498edbdef86d43e5
3,Cantonments,5.578107,-0.181827,Restaurant Chez Clarisse Mama Africa,5.564269,-0.183821,African Restaurant,1556,4fd90666e4b071d9bd1f7e6f
4,Cantonments,5.578107,-0.181827,The Honeysuckle,5.568717,-0.184842,Pub,1097,4be381c32fc7d13a2e8f083a


In [124]:
getNear['Latitudes'] = getNear['Latitudes'].astype(float)
getNear['Longitudes'] = getNear['Longitudes'].astype(float)
getNear.head()

,Places,Places Latitude,Places Longitude,Found Places,Latitudes,Longitudes,Places Category,Distance,id
0,Cantonments,5.578107,-0.181827,La Villa Boutique Hotel,5.567260,-0.183356,Hotel,1219,4ea28fde0aaf711180a74c1f
1,Cantonments,5.578107,-0.181827,Fiesta Residence,5.575963,-0.185265,Modern European Restaurant,449,55ddb5a2498e3bb48ca6b2e2
2,Cantonments,5.578107,-0.181827,Bistro 22,5.571572,-0.169086,Bistro,1587,539477fc498edbdef86d43e5
3,Cantonments,5.578107,-0.181827,Restaurant Chez Clarisse Mama Africa,5.564269,-0.183821,African Restaurant,1556,4fd90666e4b071d9bd1f7e6f
4,Cantonments,5.578107,-0.181827,The Honeysuckle,5.568717,-0.184842,Pub,1097,4be381c32fc7d13a2e8f083a


In [120]:

# create map of New York using latitude and longitude values
map_ghana = folium.Map(location=[latitude, longitude], zoom_start=10)
neighborhoods= getNear;
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitudes'], neighborhoods['Longitudes'], neighborhoods['Found Places'], neighborhoods['Places']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='tomato',
        fill=True,
        fill_color='white',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ghana)  
    
map_ghana

# Conclusion